### Recursive Reasoning in Small LLMs
**An Implementation of Reasoning-based Task Decomposition Utilizing the Agentic (Tool-Calling) Abilities of Qwen3-8B**

In [4]:
import os
import re
import json
import pandas as pd

from dotenv import load_dotenv
from openai import OpenAI
from transformers import AutoTokenizer
from typing import Dict, Any, List

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

#### Custom API Call Wrapper Class

In [5]:
## Define a simple data structure for tracking results

class RecursiveCallTracker:
    def __init__(self):
        self.log = []

    def add_entry(self, current_depth: int, reasoning_tokens: int, output_tokens: int, reasoning: str, output: str, tool_calls: str):
        self.log.append({
            "current_depth": current_depth,
            "reasoning_tokens_count": reasoning_tokens,
            "output_tokens_count": output_tokens,
            "reasoning": reasoning,
            "output": output,
            "tool_calls": tool_calls
        })

    def get_log(self):
        return self.log

## Create a custom wrapper class for the api call that facilitates tool use

class Agent:
    def __init__(self, api_key: str, model_name: str):
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-8B')

    def call(self, messages: List[Dict[str, str]], tools: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Makes a single API call and returns a parsed response.
        """

        try:
            completion = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )

            output_inference = getattr(completion.choices[0].message, 'content', '').strip()
            reasoning_inference = getattr(completion.choices[0].message, 'reasoning', '').strip()
            tool_calls = getattr(completion.choices[0].message, 'tool_calls', None)

            ## TEST
            print(output_inference, '\n')
            print(reasoning_inference, '\n')
            
            return {
                "reasoning_tokens": len(self.tokenizer.encode(reasoning_inference)),
                "output_tokens": len(self.tokenizer.encode(output_inference)),
                "reasoning": reasoning_inference,
                "output": output_inference,
                "tool_calls": tool_calls
            }
        
        except Exception as e:
            print(f"An error occurred during the API call: {e}")
            return None

#### Agentic Loop & Recursive Tool Use

In [6]:
## Initialize the custom agent and tracker

#model_name = "qwen/qwen3-8b"
model_name = "qwen/qwen3-32b"

qwen_agent = Agent(api_key, model_name=model_name)
tracker = RecursiveCallTracker()

## The tool that will be called recursively

SYSTEM_PROMPT = ''' 
    You are a judicious recursive reasoning engine.

    YOU MUST CALL THE TOOL.

    Through usage of 'recursive_tool' you will intelligently perform task decomposition during reasoning, in order to manage task complexity, accuracy and context efficiency.
    By passing an array of subtasks to recursive_tool, you are able to execute reasoning in parallel about subtasks.

    Importantly, all prompts in the array of subtasks must be *completely defined* (i.e. they MUST NOT rely on context available to the parent thread, and should provide all necessary context)
    
    Please note that 'recursive_tool' is a function which accepts three arguments: 'depth', 'max_depth', and 'prompt_messages' (an array of subtasks for the forked subroutines to execute)
'''

def recursive_tool(depth: int, max_depth: int, prompt_messages: list[str], qwen_agent: Agent, tracker: RecursiveCallTracker) -> list[str]:
    """
    A tool that demonstrates a recursive call to itself via the agent.
    """

    if depth > max_depth:
        print("Recursion limit reached. Stopping.")
        return "Recursion limit reached. Stopping."

    print(f"Calling recursively at depth: {depth}")

    results = []

    # This is the recursive part: the tool calls the agent's main loop.
    ## Iteration through prompts in prompt_messages facilitates parallel decomposition

    for prompt in prompt_messages:
        result = agent_main_loop(
            task_prompt=prompt,
            current_depth=depth + 1,
            max_depth=max_depth,
            qwen_agent=qwen_agent,
            tracker=tracker
        )

        results.append(result)

    return results

## The main agent loop that orchestrates the calls

def agent_main_loop(task_prompt: str, current_depth: int, max_depth: int, qwen_agent: Agent, tracker: RecursiveCallTracker):
    # Define the tool for the agent to use
    tools = [{
        "type": "function",
        "function": {
            "name": "recursive_tool",
            "description": "A tool that can call a LM to execute a subtask.",
            "parameters": {
                "type": "object",
                "properties": {
                    "depth": {
                        "type": "integer", 
                        "description": "The current recursion depth."
                    },
                    "max_depth": {
                        "type": "integer", 
                        "description": "The maximum allowed recursion depth."
                    },
                    "prompt_messages": {
                        "type": "array", 
                        "description": "Array (possibly of length one) of strings containing fully described recursive tasks",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["depth", "max_depth", "prompt_messages"],
            }
        }
    }]
    
    # Create the messages for the API call
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": task_prompt}
    ]

    # Make the API call using our custom wrapper
    response = qwen_agent.call(messages, tools)

    if response is None:
        return "An error occurred."

    # Log the token usage and reasoning
    tracker.add_entry(
        current_depth=current_depth,
        reasoning_tokens=response["reasoning_tokens"],
        output_tokens=response["output_tokens"],
        reasoning=response["reasoning"],
        output=response["output"],
        tool_calls=str(response["tool_calls"])
    )

    # Check for a tool call
    if response["tool_calls"]:
        tool_call = response["tool_calls"][0]
        try:
            function_args = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as e:
            print(f"Error decoding tool arguments: {e}")
            return "Error decoding tool arguments."
        
        # Execute the tool
        return recursive_tool(**function_args, qwen_agent=qwen_agent, tracker=tracker)
    else:
        # If no tool is called, the recursion ends
        return response["output"]

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

#### Test

In [ ]:
# Example usage
final_result = agent_main_loop(
    task_prompt="If $f(x) = \\frac{3x-2}{x-2}$, what is the value of $f(-2) + f(-1) + f(0)$? Express your answer as a common fraction.",
    current_depth=0,
    max_depth=2,
    qwen_agent=qwen_agent,
    tracker=tracker
)

print("\nFinal Result:", final_result)
print("\n--- Token Usage and Reasoning Log ---")
import json
print(json.dumps(tracker.get_log(), indent=2))

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


To find the value of $ f(-2) + f(-1) + f(0) $ where $ f(x) = \frac{3x - 2}{x - 2} $, we calculate each term individually:

1. **Calculate $ f(-2) $:**
   - Numerator: $ 3(-2) - 2 = -8 $
   - Denominator: $ -2 - 2 = -4 $
   - Result: $ \frac{-8}{-4} = 2 $

2. **Calculate $ f(-1) $:**
   - Numerator: $ 3(-1) - 2 = -5 $
   - Denominator: $ -1 - 2 = -3 $
   - Result: $ \frac{-5}{-3} = \frac{5}{3} $

3. **Calculate $ f(0) $:**
   - Numerator: $ 3(0) - 2 = -2 $
   - Denominator: $ 0 - 2 = -2 $
   - Result: $ \frac{-2}{-2} = 1 $

**Sum the results:**
$$
2 + \frac{5}{3} + 1 = \frac{6}{3} + \frac{5}{3} + \frac{3}{3} = \frac{14}{3}
$$

**Final Answer:**
$$
\boxed{\dfrac{14}{3}}
$$ 

Okay, let's see. I need to find the value of f(-2) + f(-1) + f(0) where the function f(x) is (3x - 2)/(x - 2). Hmm. So first, maybe I should calculate each term individually. Let me start with f(-2). 

For f(-2), substitute x = -2 into the function. So the numerator becomes 3*(-2) - 2, which is -6 - 2 = -8. The denom

#### Deprecated

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import AutoTokenizer
from IPython.display import clear_output
from dataclasses import dataclass, field
from typing import Dict, List, Any

from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
from qwen_agent.llm import get_chat_model

import os
import re
import pandas as pd
import json
import logging

load_dotenv()
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

# Define Model
model = 'qwen/qwen3-8b'
#model='qwen/qwen3-32b'

In [8]:
huggingface_model = 'Qwen/Qwen3-8B'
tokenizer = AutoTokenizer.from_pretrained(huggingface_model)

def count_tokens(text):
  '''
  Count tokens in `text` using the `Qwen3-8B` tokenizer
  '''
  return len(tokenizer.encode(text))

In [ ]:
@register_tool("think_recursively")
class RecursiveTool(BaseTool):
    """
    Creates new instances of the LLM to think about specific subtasks in parallel.
    """

    description = ''' 
        Recursive reasoning tool.
        This tool takes 3 arguments; current recursion depth, maximum recursion depth and an array of self-contained subtasks
        As output the tool returns an array containing the answers of the subthreads executed on their relevant subtasks
    '''

    parameters = {
        "type": "object",
        "properties": {
            "depth": {
                "type": "integer", 
                "description": "The current recursion depth."
            },
            "max_depth": {
                "type": "integer", 
                "description": "The maximum allowed recursion depth."
            },
            "prompt_messages": {
                "type": "array", 
                "description": "Array (possibly of length one) of strings containing fully described recursive tasks",
                "items": {
                    "type": "string"
                }
            },
        },
        "required": ["depth", "max_depth", "prompt_messages"],
    }

    def call(self, params: str, **kwargs) -> str:
        curr_depth, max_depth, task_prompts = json.loads(params)['depth'], json.loads(params)['max_depth'], json.loads(params)['prompt_messages']
        print(params)
        return "Successfully utilized the tool call!"
        # try:
        #     args = json.loads(params)
        #     subtasks = args.get("subtasks", [])

        #     if level >= self._max_depth:
        #         return f"Max depth ({self._max_depth}) reached. Subtasks not processed."

        #     results = []
        #     for subtask in subtasks:
        #         recursive_agent = QwenAgentRecursiveTool(
        #             api_key=self._api_key,
        #             model=self._model,
        #             max_depth=self._max_depth,
        #             parent_token_stats=self._token_stats
        #         )
                
        #         response = recursive_agent._recursive_reason(subtask, level + 1)
                
        #         results.append(f"Result for subtask '{subtask}': {response.output}")

        #     return "\n\n".join(results)
        
        # except (json.JSONDecodeError, KeyError) as e:
        #     return f"Error in tool arguments: {str(e)}"

In [19]:
llm_cfg = {
    'model': 'qwen/qwen3-coder:free',
    'model_server': "https://openrouter.ai/api/v1",
    'api_key': openrouter_api_key,
    'stream': False,
    'reasoning': {
        'effort': 'high',
        'max_tokens': 2000,
        'enabled': True,
        'exclude': False
    }
}

llm = get_chat_model(llm_cfg)

tools = []# ['think_recursively']

agent = Assistant(llm=llm_cfg, function_list=tools)

system_prompt = '''
    Solve this problem
'''

task = "Find the sum of all integer bases $b>9$ for which $17_b$ is a divisor of $97_b.$"

messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        'role': 'user',
        'content': task
    }
]

responses = []
for response in agent.run(messages=messages):
    responses.append(response)

print(responses[-1])

[{'role': 'assistant', 'content': 'I need to find all integer bases $b > 9$ for which $17_b$ divides $97_b$.\n\nFirst, let me convert these base-$b$ numbers to base 10.\n\n$17_b = 1 \\cdot b + 7 = b + 7$\n\n$97_b = 9 \\cdot b + 7 = 9b + 7$\n\nFor $17_b$ to divide $97_b$, we need $(b + 7)$ to divide $(9b + 7)$.\n\nLet me use polynomial division or rewrite this divisibility condition.\n\nI want to find when $(b + 7) | (9b + 7)$.\n\nI can rewrite $9b + 7$ in terms of $(b + 7)$:\n$9b + 7 = 9(b + 7) - 9 \\cdot 7 + 7 = 9(b + 7) - 63 + 7 = 9(b + 7) - 56$\n\nSo $(b + 7) | (9b + 7)$ if and only if $(b + 7) | 56$.\n\nThis means $b + 7$ must be a divisor of 56.\n\nLet me find the divisors of 56:\n$56 = 2^3 \\cdot 7 = 8 \\cdot 7$\n\nThe positive divisors of 56 are: 1, 2, 4, 7, 8, 14, 28, 56\n\nSo $b + 7 \\in \\{1, 2, 4, 7, 8, 14, 28, 56\\}$\n\nThis gives us:\n$b \\in \\{-6, -5, -3, 0, 1, 7, 21, 49\\}$\n\nSince we need $b > 9$, the valid values are $b = 21$ and $b = 49$.\n\nLet me verify these solu

In [18]:
print(json.dumps(responses, indent=2))

[
  [
    {
      "role": "assistant",
      "content": "I"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer bases"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer bases $"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer bases $b"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer bases $b >"
    }
  ],
  [
    {
      "role": "assistant",
      "content": "I need to find all integer bases $b > "
    }
  ],


In [23]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

messages = [{"role": "user", "content": "Find the sum of all integer bases $b>9$ for which $17_b$ is a divisor of $97_b.$"}]

completion = client.chat.completions.create(
    model='qwen/qwen3-8b',
    messages=messages,
    stream=False
)

print(completion.choices[0].message)

KeyboardInterrupt: 